![Quantinuum_logo](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAIIAAABLCAMAAACLOq%2BzAAACx1BMVEUAAAAAAP%2BAgIBVqqqAgIBmmZmAgKptkpKAgJ9xjo6AgJl0i6JqgJV2iZ1tgJJ3iJlwgJ94h5ZxgJx5hpRzgJl5hp50gJdvhZt1gJVwhZl2gJ12gJtyhJV3gJlzhJx4gJd0g5txgJZ1g5lxgJx1g5hygJp2g5ZzgJl2g5tzgJh3gpp0gJdxgpl0gJtygph1gJpygpd1gJlzgpt2gJhzgpp2gJd0gplygJt0gphygJp1gpdzgJl1gptzgJh1gpp2gZl0gJtygZh0gJpzgZh1gJlzgZp1gJhzgZp1gJh0gZl0gZhygJp0gZhzgJl0gZpzgJh1gZpzgJh1gZl0gJp1gZh0gJp0gJlzgZp0gJhzgZp1gJhzgZl1gJp0gZh1gJp0gZhzgJl0gZpzgJlzgJh1gZlzgJp1gZl1gZh0gJl0gJlzgZl0gJhzgZl1gJp0gZl1gJl0gZh1gJl0gZpzgJl0gZlzgJh0gZlzgJp0gZl0gJl1gJh1gJpzgJl0gJhzgJl0gJpzgJl0gJl1gJl0gJp1gJl0gJl1gJhzgJp0gJlzgJl0gJh0gJl1gJp0gJl1gJl0gJh1gJl0gJpzgJl0gJlzgJh0gJl0gJp0gJl0gJl1gJh0gJl1gJp0gJlzgJl0gJh0gJl0gJp0gJl0gJl0gJh1gJl0gJpzgJh0gJl0gJl0gJl1gJp0gJl1gJl0gJhzgJl0gJl0gJl0gJh0gJl0gJp0gJl0gJl1gJh0gJlzgJp0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJlzgJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJlzgJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJn%2F%2F%2F%2Bc0JCdAAAA63RSTlMAAQIDBAUGBwgJCgsMDQ4PEBESExQVFhcYGRocHR4fICEiIyQlJicoKSorLC0uLzAxMjM0NTY3ODk6Ozw9Pj9BQkNERUZHSElKS01OT1BRUlNUVVZXWFpbXF1eX2BhYmNkZWZoaWprbW5wcXJzdHV2d3h5ent8fX5%2FgIGDhYaHiImKjI2Oj5CSk5SVlpeYmZqbnJ2en6ChoqOkpaanqKmqq6ytrq%2Bws7a3ubq7vL2%2BwMHCw8TFxsfIycrLzM3Oz9DR0tPU1dbY2drb3N3e3%2BDh4uPk5ebn6Orr7O3u7%2FHy8%2FT19vf4%2Bfr7%2FP3%2B7Jw5zgAAAAFiS0dE7CG5sxsAAAVfSURBVBgZ7cGNV9X1AcfxNxcwnhRRaVCsKGszI%2BbDcs3KlDKldAmhGywz0ZyJOqLUHrBraVoKJD5E%2BBDaarXS2tZ0jmkqKjlLKR9SA6EFCNzPP7Hv7%2F7QmMm9nM754jkdXi9%2BsNCxL7x9uLb266qKZx%2BI5Ar4yYsn9J2GNyd46F6e%2BfW6RFWmh2404ANdxq6f022uOajL%2BvYPIXSPyN3qTFkY1vUBytW5tyOwLHE%2BjFMg73iwKqpyCCEHFFAhVpXsgfEKzJeBRWO1CDYpiK%2FisCbsU91N%2BDkFU4Q1WVIkqWrX%2BvGTv04Ih96Dp246o47aUrBlpxogR351z8XzHc%2F4j9TBGiz5lVQD82W0Lo7lErfv0EWN8dixTDoICyTV3Mn3heY36YK52LFf%2BgLypH%2FHcVlDT6vddqxI8EnfQLb29qMTg0%2FJ1RSNDQ%2FJiGJEVTydGlQv1%2F3YMEdGGp4BBJAt1xPYsFTGYoLYIL8l2LBZRhVBJDXJ8TI2%2FEmOewhilRwrsGGDHO8QxI1yvIgNa%2BQ3hiCqZCzChmXyq76KwBbLmIsN0%2BQqIbAHZEzGhmFql0dAKTKSsCGiRa622QQSI6kGO7bpgqWhBNAgrcSOabroX7fRuQ8%2Fqx6JHX2bdNH55ddxJZSrg5aNE6Lpdre06v80bnvVW5DfbnZuR6OxZKW6qhxL%2Btaoi97Altub1TWvY81jPnXJeuzJV5esw6LHW9UFa7Ep%2FayCK8Wqaz9QUKux7OFjCqIE28If2aeAiukGtxX%2B47w6VUT3iBg8Mdc1K%2F8S99GjR48ePXr06PHjkzRjycIxHhyR0%2BNwxOT0wxH64G%2F74xrzaDR%2B9%2F4SeDDzghDwZCZh3PdoJH7jhgLXZobhiH84Aph4E36TBgGpmem0S52eDuFLz8vYPwRjiMbj%2BJlycGRLr%2BKq1N964zi0HqjXBTEQockYB7UtGsexImCSYnHMVCJQl4cjwrcAWCaNxy%2F2tE4SskXb74i%2B5omGhhRgpDJwpGgGRtjhszuarsNvd5t2xmJ8XgYMHZ2WdqQuLS1tWBhEaQrGoTb9tTfGVyXAJF2NI0%2BJQF0Bjr5aACz31X%2FaC8cSfXOWDK0LwfjFt7tC4E49gmOEpmFMVW5KSyl%2Be%2F68QJX9gFPluHafwBWj32FUbynUjljg3GrgIf0UR74SgbrncAzQ08CKs7M0DyO56d2t9Ww%2FFYPfUxoOqZqN435lAeGf7fVQ0jIQx97NzNG%2BBKjbiKvyGK4%2BysE4vJYC7Y6H5lJgggbheF6JwJmXcFyvfOCVk6H7ziUAFa23ljdzeg2uFM2CG7QQxxSNA6ZrEiS3lOPYvwHm6EgyjZtx7TqCq69%2Bj3FkNTyp6iSP1gLpugvHMiUCp9bjSFEesPILRqkM7tFrlLbS8Bqu6%2FVHCP36n6EYmxv7QK9jenxKVu6J1sEYB8qAuTpxq68C18fVuOI0FePzYqBANalaDyRrHUbvmuO9gA%2FPxGE8rVFA0VHY4rsr7EBDAkUtHNiJK10ZwHy9lUD0cnmBmWq3FeNQGUaBarUV1%2FYqXP01DeNoMcYi1aoMY5MKI0neoRyMe31%2Fv5FeeS0fhQAlR2Fg8755WgirmnlWo%2FCLHRgKeFZIdY16Ixyu%2BrJy%2BDBHqW8IUF2G4xnpLVzvfYIrXtMxaopxPC%2BVY8T8ReePy1eI38xmX81%2FtedqjNVHgZfV9mUUrGok%2FvjpsXQ0svj90jSMG7zD8Yt%2BZiwwLwu%2FXG82rqzHcMV4R2AU%2FAa%2FGd7JOEImvv7e8mG0u3nJuxuzw3Fk5AP9vd7RwMQCGPQfHayoKClZ6R3FlRJV8MnJ2trG2trZXBH%2FA2RBOeTINIkQAAAAAElFTkSuQmCC)<br>
# Submitting to Quantinuum Emulators via pytket

This notebook contains examples for running quantum circuits on Quantinuum's emulators.<br><br>
We can use an emulator to get an idea of what a quantum device might output for our quantum circuit. This enables circuit debugging and optimization before running on a physical machine.<br><br>
See the *Quantinuum Systems User Guide* on the Quantinuum User Portal for detailed information on each of the systems available and workflow information including job submission, queueing, and data retention.<br>
* [Basic Example](#example)<br>
* [Without Noise](#no-noise)<br>
* [Stabilizer Emulator](#stabilizer)

## Basic Example <a class="anchor" id="example"></a>

### Circuit Preparation

Create your circuit via the pytket python library. For details on getting started with `pytket`, see pytket's [Getting Started](https://cqcl.github.io/tket/pytket/api/getting_started.html) page.

In [ ]:
from pytket.circuit import Circuit, fresh_symbol
from pytket.circuit.display import render_circuit_jupyter

Set up Bell Test

In [ ]:
circuit = Circuit(2, name="Bell Test")
circuit.H(0)
circuit.CX(0, 1)
circuit.measure_all()
render_circuit_jupyter(circuit)

### Select Device

*Available emulators:*<br>
- `H1-1E`, `H1-2E`: Device-specific Emulators for H1-1 and H1-2

Login to the Quantinuum API using your credentials and check the device status.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend

In [ ]:
machine = "H1-2E"
backend = QuantinuumBackend(device_name=machine)
backend.login()

In [ ]:
print(machine, "status:", QuantinuumBackend.device_state(device_name=machine))

### Circuit Compilation

`pytket` includes many features for optimizing circuits. This includes reducing the number of gates where possible and resynthesizing circuits for a quantum computer's native gate set. See the `pytket` [User Manual](https://cqcl.github.io/pytket/manual/index.html) for more information on all the options that are available.<br><br>
Here the circuit is compiled with `get_compiled_circuit`, which includes optimizing the gates and resynthesizing the circuit to Quantinuum's native gate set. The `optimisation_level` sets the level of optimization to perform during compilation, check pytket documentation for more details.

In [ ]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=1)
render_circuit_jupyter(compiled_circuit)

### Run the Circuit

Now the circuit can be run on an emulator.

In [ ]:
n_shots = 100
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)

Check the status of the job.

In [ ]:
status = backend.circuit_status(handle)
print(status)

### Retrieve Results

Once a job's status returns completed, results can be returned using the `get_result` function.

In [ ]:
result = backend.get_result(handle)
result

### Save Results

It is recommended that users save job results as soon as jobs are completed due to the Quantinuum data retention policy.

In [ ]:
import json

In [ ]:
with open("pytket_emulator_example.json", "w") as file:
    json.dump(result.to_dict(), file)

### Analyze Results

The result output is just like that of a quantum device.<br><br>
The simulation by default runs with noise; therefore, it won't give back all `11`'s.

In [ ]:
result = backend.get_result(handle)
print(result.get_distribution())

In [ ]:
print(result.get_counts())

## Without Noise <a class="anchor" id="no-noise"></a>

The Quantinuum emulators may be run with or without the physical device noise model. We can set `noisy_simulation=False` to do this.

In [ ]:
n_shots = 100
no_error_model_handle = backend.process_circuit(
    compiled_circuit, n_shots=n_shots, noisy_simulation=False
)
print(no_error_model_handle)

In [ ]:
no_error_model_status = backend.circuit_status(no_error_model_handle)
print(no_error_model_status)

In [ ]:
no_error_model_result = backend.get_result(no_error_model_handle)
no_error_model_result

In [ ]:
with open("pytket_emulator_no_error_model_example.json", "w") as file:
    json.dump(result.to_dict(), file)

In [ ]:
no_error_model_result = backend.get_result(no_error_model_handle)
print(no_error_model_result.get_distribution())
print(no_error_model_result.get_counts())

## Stabilizer Emulator <a class="anchor" id="stabilizer"></a>

By default, emulations are run using a state-vector emulator, which simulates any quantum operation. However, if the quantum operations are all Clifford gates, it can be faster for complex circuits to use the `stabilizer` emulator. The stabilizer emulator is requested in the setup of the `QuantinuumBackend` with the `simulator` input option. This only applies to Quantinuum emulators.

In [ ]:
machine = "H1-2E"
stabilizer_backend = QuantinuumBackend(device_name=machine, simulator="stabilizer")
print(machine, "status:", QuantinuumBackend.device_state(device_name=machine))
print("Simulation type:", stabilizer_backend.simulator_type)

In [ ]:
n_shots = 100
stabilizer_handle = stabilizer_backend.process_circuit(
    compiled_circuit, n_shots=n_shots
)
print(stabilizer_handle)

In [ ]:
stabilizer_status = stabilizer_backend.circuit_status(stabilizer_handle)
print(stabilizer_status)

In [ ]:
stabilizer_result = stabilizer_backend.get_result(stabilizer_handle)
stabilizer_result

In [ ]:
with open("pytket_emulator_stabilizer_example.json", "w") as file:
    json.dump(result.to_dict(), file)

In [ ]:
stabilizer_result = stabilizer_backend.get_result(stabilizer_handle)
print(stabilizer_result.get_distribution())
print(stabilizer_result.get_counts())

<div align="center"> &copy; 2022 by Quantinuum. All Rights Reserved. </div>